In [2]:
#import CGIHTTPServer
#import SimpleHTTPServer
from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
#from http.server import BaseHTTPRequestHandler, HTTPServer
import sqlite3 as lite
import json
con = lite.connect('server_fin.db')
cursor = con.cursor()

PORT = 5000

Handler = BaseHTTPRequestHandler

class HttpProcessor(BaseHTTPRequestHandler):
    

    def do_GET(self):
        try:
            args =  self.path.split('?')
            if (args[0] == '/post_top_comments'):
                self.post_top_comments(args[1])
            if (args[0] == '/post_title'):
                self.post_title(args[1])
            if (args[0] == '/post_author'):
                self.post_author(args[1])
            if (args[0] == '/post_date'):
                self.post_date(args[1])
            if (args[0] == '/date_top_comments'):
                self.date_top_comments(args[1])
            if (args[0] == '/user_top_comments'):
                self.user_top_comments(args[1])
        except Exception:
            self.send_response(200)
            self.send_header('content-type','text/html')
            self.end_headers()
            self.wfile.write("error")


        #self.send_response(200)
        #self.send_header('content-type','text/html')
        #self.end_headers()
        #self.wfile.write("hello !")

    def post_title(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT title FROM posts WHERE id=?", [(id)])
        title = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'title' : title })
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)


    def post_author(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT author FROM posts WHERE id=?", [(id)])
        author = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'author': author})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)

    def post_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        p_id = int(a[0].split('=')[1])
        cursor.execute("SELECT rate, date, post_id, id, time FROM comments WHERE post_id=?", [(p_id)])
        comments = cursor.fetchall()
        comments.sort(key=lambda tup: (-tup[0], tup[1], tup[4]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[2]) + '/#comment_' + str(i[3]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)


    def post_date(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT date FROM posts WHERE id=?", [(id)])
        date = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'date': date})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)

    def date_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        date = a[0].split('=')[1]
        cursor.execute("SELECT rate, post_id, id, time FROM comments WHERE date=?", [(date)])
        comments = cursor.fetchall()
        comments.sort(key=lambda tup: (-tup[0], tup[3]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[1]) + '/#comment_' + str(i[2]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)


    def user_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        name = a[0].split('=')[1]
        cursor.execute("SELECT rate, date, post_id, id, time FROM comments WHERE author=?", [(name)])
        comments = cursor.fetchall()
        comments.sort(key = lambda tup : (-tup[0], tup[1], tup[4]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[2])+'/#comment_'+str(i[3]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print(json_string)


In [3]:
#proc = HttpProcessor()
#proc.connect_db(cursor)
print(cursor.fetchall())
serv = HTTPServer(("127.0.0.1",PORT), HttpProcessor)
serv.serve_forever()

[]


127.0.0.1 - - [20/May/2018 05:11:39] "GET /post_date?post_id=201576 HTTP/1.1" 200 -


{"date": "2013-11-10"}


127.0.0.1 - - [20/May/2018 05:14:40] "GET /post_top_comments?post_id=247319&n=5 HTTP/1.1" 200 -


{"comments": ["247319/#comment_9164430", "247319/#comment_9164448", "247319/#comment_9164442", "247319/#comment_9164446", "247319/#comment_9164432"]}


...

127.0.0.1 - - [20/May/2018 05:30:00] "GET /user_top_comments?username=zee&n=5 HTTP/1.1" 200 -


{"comments": ["234663/#comment_7906781", "229431/#comment_7766665", "222855/#comment_7588523", "229981/#comment_7781643", "222855/#comment_7589071"]}


127.0.0.1 - - [20/May/2018 05:30:43] "GET /date_top_comments?date=2016-04-01&n=5 HTTP/1.1" 200 -


{"comments": ["280664/#comment_8834074", "280682/#comment_8834860", "280610/#comment_8835510", "280682/#comment_8834796", "280660/#comment_9686278"]}


KeyboardInterrupt: 

In [3]:
#import CGIHTTPServer
#import SimpleHTTPServer
from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
import sqlite3 as lite
import json
con = lite.connect('server_fin.db')
cursor = con.cursor()

PORT = 6780

Handler = BaseHTTPRequestHandler

class HttpProcessor(BaseHTTPRequestHandler):
    

    def do_GET(self):
        try:
            args =  self.path.split('?')
            if (args[0] == '/post_top_comments'):
                self.post_top_comments(args[1])
            if (args[0] == '/post_title'):
                self.post_title(args[1])
            if (args[0] == '/post_author'):
                self.post_author(args[1])
            if (args[0] == '/post_date'):
                self.post_date(args[1])
            if (args[0] == '/date_top_comments'):
                self.date_top_comments(args[1])
            if (args[0] == '/user_top_comments'):
                self.user_top_comments(args[1])
        except Exception:
            self.send_response(200)
            self.send_header('content-type','text/html')
            self.end_headers()
            self.wfile.write("error")


        #self.send_response(200)
        #self.send_header('content-type','text/html')
        #self.end_headers()
        #self.wfile.write("hello !")

    def post_title(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT title FROM posts WHERE id=?", [(id)])
        title = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'title' : title })
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string


    def post_author(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT author FROM posts WHERE id=?", [(id)])
        author = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'author': author})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string

    def post_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        p_id = int(a[0].split('=')[1])
        cursor.execute("SELECT rate, date, post_id, id, time FROM comments WHERE post_id=?", [(p_id)])
        comments = cursor.fetchall()
        comments.sort(key=lambda tup: (-tup[0], tup[1], tup[4]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[2]) + '/#comment_' + str(i[3]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string


    def post_date(self, args):
        id = int(args.split('=')[1])
        cursor.execute("SELECT date FROM posts WHERE id=?", [(id)])
        date = (cursor.fetchall()[0])[0]
        json_string = json.dumps({'date': date})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string

    def date_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        date = a[0].split('=')[1]
        cursor.execute("SELECT rate, post_id, id, time FROM comments WHERE date=?", [(date)])
        comments = cursor.fetchall()
        comments.sort(key=lambda tup: (-tup[0], tup[3]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[1]) + '/#comment_' + str(i[2]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string


    def user_top_comments(self, args):
        a = args.split('&')
        n = int(a[1].split('=')[1])
        name = a[0].split('=')[1]
        cursor.execute("SELECT rate, date, post_id, id, time FROM comments WHERE author=?", [(name)])
        comments = cursor.fetchall()
        comments.sort(key = lambda tup : (-tup[0], tup[1], tup[4]), reverse=False)
        res = []
        for i in comments[:n]:
            res.append(str(i[2])+'/#comment_'+str(i[3]))
        json_string = json.dumps({'comments': res})
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        self.wfile.write(json_string)
        print json_string



#proc = HttpProcessor()
#proc.connect_db(cursor)
print cursor.fetchall()
serv = HTTPServer(("127.0.0.1",PORT), HttpProcessor)
serv.serve_forever()

[]


127.0.0.1 - - [20/May/2018 16:27:51] "GET /post_date?post_id=229391 HTTP/1.1" 200 -


{"date": "2014-07-10"}


127.0.0.1 - - [20/May/2018 16:28:07] "GET /post_author?post_id=280614 HTTP/1.1" 200 -


{"author": "simonsson"}


...

127.0.0.1 - - [20/May/2018 16:28:39] "GET /date_top_comments?date=2016-04-01&n=5 HTTP/1.1" 200 -


{"comments": ["280664/#comment_8834074", "280682/#comment_8834860", "280610/#comment_8835510", "280682/#comment_8834796", "280660/#comment_9686278"]}


127.0.0.1 - - [20/May/2018 16:28:40] "GET /user_top_comments?username=zee&n=5 HTTP/1.1" 200 -


{"comments": ["234663/#comment_7906781", "229431/#comment_7766665", "222855/#comment_7588523", "229981/#comment_7781643", "222855/#comment_7589071"]}


KeyboardInterrupt: 